In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import clip_grad_norm_
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, f1_score, accuracy_score
from tqdm import tqdm_notebook as tqdm
import torchnet as tnt
from nltk import tokenize
#from keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings('ignore')
from utils import *
import pickle
import matplotlib.pyplot as plt
%matplotlib inline 
from models import *
import string

Using CNTK backend


using gpu


In [2]:
stopwords = []
word2vec, vecs = load_word2vec('./data/glove.6B.100d.txt', stopwords)
word2id = dict(zip(word2vec.keys(), range(1, len(word2vec) + 1)))

pickle.dump(word2id, open('./tmp/word2id_all_words.pkl', 'wb'))
pickle.dump(vecs, open('./tmp/pretrained_all_words.pkl', 'wb'))

In [3]:
train_bodies = pd.read_csv('./data/train_bodies.csv')
train = pd.read_csv('./data/train_stances.csv')

df_train = train.merge(train_bodies, on=['Body ID'], how='left')
df_train.head()

test_bodies = pd.read_csv('./data/competition_test_bodies.csv')
test = pd.read_csv('./data/competition_test_stances.csv')

df_test = test.merge(test_bodies, on=['Body ID'], how='left')
df_test.head()

train_ids = [int(v) for v in open('./data/training_ids.txt').read().strip().split('\n')]
val_ids = [int(v) for v in open('./data/hold_out_ids.txt').read().strip().split('\n')]
        
df_train_a = df_train[df_train['Body ID'].isin(train_ids)]
df_train_b = df_train[df_train['Body ID'].isin(val_ids)]
print(df_train_a.shape)
print(df_train_b.shape)
print(df_train.shape)
print(df_test.shape)

(40350, 4)
(9622, 4)
(49972, 4)
(25413, 4)


In [4]:
train_headlines, train_bodies = to_ids(df_train_a, word2id)
label_2_id = {'agree': 0, 'disagree': 1, 'discuss':2, 'unrelated': 3}
pickle.dump({'h': train_headlines, 'b': train_bodies, 'y': df_train_a['Stance'].map(lambda x: label_2_id[x]).values}, 
            open('./tmp/train_ids_all_words.pkl', 'wb'))

val_headlines, val_bodies = to_ids(df_train_b, word2id)
pickle.dump({'h': val_headlines, 'b': val_bodies, 'y': df_train_b['Stance'].map(lambda x: label_2_id[x]).values}, 
            open('./tmp/val_ids_all_words.pkl', 'wb'))

test_headlines, test_bodies = to_ids(df_test, word2id)
label_2_id = {'agree': 0, 'disagree': 1, 'discuss':2, 'unrelated': 3}
pickle.dump({'h': test_headlines, 'b': test_bodies, 'y': df_test['Stance'].map(lambda x: label_2_id[x]).values}, 
            open('./tmp/test_ids_all_words.pkl', 'wb'))